
****

***
# <font color=red size=10>Modelo de Classificação para Churn</font>
***

****

Atualmente, uma empresa enfrenta um desafio significativo relacionado a um aumento no número de churn, com muitos clientes cancelando o serviço. Para enfrentar essa situação, o time de marketing propôs uma estratégia proativa: oferecer promoções direcionadas aos clientes que apresentam maior probabilidade de cancelamento, buscando assim evitar a perda desses clientes valiosos.

Então, será desenvolvido um modelo de Machine Learning. Este modelo será construído com base em dados históricos da empresa, que incluem informações sobre os clientes, seus contratos e os eventos de cancelamento anteriores. Essa abordagem nos permitirá classificar os clientes em categorias de possíveis cancelamentos ou não, proporcionando insights valiosos para direcionar as ações de retenção da empresa com mais eficácia.

#Iniciando a SparkSession

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=681b5ee45c8f410cdf2b1da2f149db4cc9f0f79bc4069333b36df49d436d37c9
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [177]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.master('local[*]').appName('Modelo de Classificaçao para Ocorrência de Cancelamento').getOrCreate()

In [5]:
spark

#Carregando os Dados

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
dados = spark.read.csv('/content/drive/MyDrive/Data_/dados_clientes.csv',
                      sep = ',', header  = True, inferSchema = True)

In [9]:
dados.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|   SegurancaOnline|      BackupOnline| SeguroDispositivo|    SuporteTecnico|           TVaCabo|   StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|  0|  Nao|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|               Nao|               Sim|              

In [10]:
dados.count()

10348

In [12]:
dados.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



Como podemos observar, a base de dados tem um número igual de elementos para cada uma das labels, dessa forma, podemos entender que **a base está balanceada.**

In [13]:
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



A maioria dos dados se apresenta como **string**, algo que numa base de dados comum não há problemas, mas no nosso caso, é. **Isso porque precisamos dos dados como números** para podermos utilizar um modelo de classificaçao corretamente na base. Para isso, iremos tratar a base no próximo tópico

#Tratando os dados

In [19]:
colunasStr = [
    'Churn',
    'Conjuge',
    'Dependentes',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'BackupOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [16]:
from pyspark.sql import functions as f

In [20]:
colunasTrans = [f.when(f.col(c) =='Sim',1).otherwise(0).alias(c) for c in colunasStr ]

In [21]:
for coluna in reversed(dados.columns):
  if coluna not in colunasStr:
    colunasTrans.insert (0, coluna)

colunasTrans

['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorr

In [23]:
dados.select(colunasTrans).show()

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|              1|        DSL| Mensalmente|BoletoEletronico|        29.85|    0|      1|          0|           0|                       0|              0|           1|                0|             0|      0|              0|      

In [27]:
dados_new =  dados.select(colunasTrans)

##Dummies

In [28]:
dados_new.select(['Internet', 'TipoContrato', 'MetodoPagamento']).show()

+-----------+------------+----------------+
|   Internet|TipoContrato| MetodoPagamento|
+-----------+------------+----------------+
|        DSL| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|          Boleto|
|        DSL| Mensalmente|          Boleto|
|        DSL|       UmAno|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|   CartaoCredito|
|        DSL| Mensalmente|          Boleto|
|FibraOptica| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|   DebitoEmConta|
|        DSL| Mensalmente|          Boleto|
|        Nao|    DoisAnos|   CartaoCredito|
|FibraOptica|       UmAno|   CartaoCredito|
|FibraOptica| Mensalmente|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica|    DoisAnos|   CartaoCredito|
|        Nao|       UmAno|          Boleto|
|FibraOptica|    DoisAnos|   DebitoEmConta|
|        DSL| Mensalmente|   CartaoCredito|
|FibraOptica| Mensalmente|Boleto

In [32]:
dados_new.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0).show()

+----+---+-----------+---+
|  id|DSL|FibraOptica|Nao|
+----+---+-----------+---+
|7982|  1|          0|  0|
|9465|  0|          1|  0|
|2122|  1|          0|  0|
|3997|  1|          0|  0|
|6654|  0|          1|  0|
|7880|  0|          1|  0|
|4519|  0|          1|  0|
|6466|  0|          1|  0|
| 496|  1|          0|  0|
|7833|  0|          1|  0|
|1591|  0|          0|  1|
|2866|  0|          1|  0|
|8592|  0|          1|  0|
|1829|  0|          1|  0|
| 463|  0|          1|  0|
|4900|  0|          1|  0|
|4818|  0|          1|  0|
|7554|  1|          0|  0|
|1342|  0|          0|  1|
|5300|  0|          1|  0|
+----+---+-----------+---+
only showing top 20 rows



In [33]:
Internet = dados_new.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0)
TipoContrato = dados_new.groupBy('id').pivot('TipoContrato').agg(f.lit(1)).na.fill(0)
MetodoPagamento = dados_new.groupBy('id').pivot('MetodoPagamento').agg(f.lit(1)).na.fill(0)

In [35]:
dadosD = dados_new\
          .join(Internet, 'id', how = 'inner')\
          .join(TipoContrato,'id', how = 'inner')\
          .join(MetodoPagamento, 'id', how = 'inner')\
          .select('*',
              f.col('DSL').alias('Internet_DSL'),
              f.col('FibraOptica').alias('Internet_FibraOptica'),
              f.col('Nao').alias('Internet_Nao'),
              f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
              f.col('UmAno').alias('TipoContrato_UmAno'),
              f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
              f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'),
              f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
              f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
              f.col('Boleto').alias('MetodoPagamento_Boleto')
            )\
            .drop('Internet', 'TipoContrato', 'MetodoPagamento', 'DSL',
            'FibraOptica', 'Nao', 'Mensalmente', 'UmAno', 'DoisAnos',
            'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto')



In [36]:
dadosD.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContr

#Preparação para o Modelo de Classificação

In [37]:
dadosD.show()

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [38]:
from pyspark.ml.feature import VectorAssembler

In [40]:
dadosD = dadosD.withColumnRenamed('Churn', 'label')

In [44]:
X = dadosD.columns

In [45]:
X.remove('label')
X.remove('id')

In [46]:
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [48]:
assembler = VectorAssembler(inputCols = X, outputCol = 'features' )

In [50]:
dadosF = assembler.transform(dadosD).select('features','label')

In [51]:
dadosF.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(24,[1,2,11,12,13...|    1|
|(24,[1,2,3,5,6,8,...|    1|
|(24,[1,2,5,6,10,1...|    0|
|(24,[1,2,3,5,8,12...|    0|
|(24,[1,2,3,5,6,11...|    1|
|(24,[1,2,5,6,12,1...|    1|
|(24,[1,2,3,5,6,8,...|    0|
|(24,[1,2,5,6,15,1...|    0|
|(24,[1,2,3,5,7,8,...|    0|
|(24,[1,2,3,5,12,1...|    1|
|(24,[1,2,5,16,17,...|    0|
|(24,[1,2,5,8,12,1...|    0|
|(24,[1,2,3,5,6,11...|    1|
|(24,[1,2,5,6,13,1...|    0|
|(24,[1,2,5,6,8,11...|    1|
|(24,[0,1,2,3,5,6,...|    1|
|(24,[0,1,2,3,5,7,...|    0|
|(24,[1,2,5,8,14,1...|    1|
|(24,[1,2,5,16,19,...|    0|
|(24,[1,2,3,4,5,11...|    1|
+--------------------+-----+
only showing top 20 rows



In [52]:
seed = 124
treino, teste = dadosF.randomSplit([0.7,0.3], seed = seed)

In [53]:
treino.count()

7253

In [54]:
teste.count()

3095

#Modelo de Classificação

##Regressão Logística


O primeiro modelo supervisionado que iremos usar é a **regressão logística**, que  prevê a probabilidade de uma variável categórica com base em uma ou mais variáveis independentes em base numa sigmoide de intervalo de 0 a 1.



In [55]:
from pyspark.ml.classification import LogisticRegression

In [56]:
lr = LogisticRegression()

In [57]:
modelo_lr = lr.fit(treino)

In [129]:
predito_lr = modelo_lr.transform(treino)

In [130]:
predito_lr_teste = modelo_lr.transform(teste)

In [131]:
predito_lr_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[0.75479638722921...|[0.68022290853450...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.81958998442710...|[0.69414929825143...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[1.34236042990283...|[0.79287784528464...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[0.26414706491728...|[0.56565545756733...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-1.3261368054818...|[0.20979910087761...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.2268226481841...|[0.44353621212617...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[-0.0877473148439...|[0.47807723584452...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[0.00257218401401...|[0.50064304564896...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[4.35976419395104...|[0.98737990131881...|       0.0|
|(24,[0,1,2,3,4,

##Árvore de Decisão

O segundo modelo que usaremos é a **árvore de decisão** -  um modelo de aprendizado de máquina que toma decisões com base em condições simples sobre as características dos dados. Ela é chamada de "árvore" porque se assemelha a uma estrutura de árvore invertida, com um nó raiz, nós internos representando características e nós folha representando as decisões ou resultados.

In [100]:
from pyspark.ml.classification import DecisionTreeClassifier

In [105]:
dtc = DecisionTreeClassifier(seed = seed, maxDepth = 5)

In [106]:
modelo_dtc = dtc.fit(treino)

In [116]:
predito_dtc = modelo_dtc.transform(treino)
predito_dtc.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2072.0,318.0]|[0.86694560669456...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[2072.0,318.0]|[0.86694560669456...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [24.0,5.0]|[0.82758620689655...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[2072.0,318.0]|[0.86694560669456...|       0.0|
|(24,[0,1,2,3,4,5,...|    0| [106.0,165.0]|[0.39114391143911...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|    [24.0,5.0]|[0.82758620689655...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[425.0,2103.0]|[0.16811708860759...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[425.0,2103.0]|[0.16811708860759...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[425.0,2103.0]|[0.16811708860759...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[425.0,2103.0]|[0.16811708860759...|       1.0|
|(24,[0,1,2,

In [128]:
predito_dtc_teste = modelo_dtc.transform(teste)

##Random Forest

O terceiro modelo que usaremos é o **random forest classifier**, que combina múltiplas árvores de decisão para fazer previsões mais precisas. Cada árvore é treinada em uma subamostra aleatória dos dados, e as previsões finais são obtidas através da votação das previsões individuais das árvores

In [136]:
from pyspark.ml.classification import RandomForestClassifier

In [139]:
rfc = RandomForestClassifier(seed = seed, maxDepth = 5, numTrees=10)
modelo_rfc = rfc.fit(treino)

In [141]:
predito_rfc = modelo_rfc.transform(treino)
predito_rfc.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[8.20674462409238...|[0.82067446240923...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[7.65056169137184...|[0.76505616913718...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[3.11833301758737...|[0.31183330175873...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[7.96711826649231...|[0.79671182664923...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[2.53509894968981...|[0.25350989496898...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[3.11833301758737...|[0.31183330175873...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[4.23257826589132...|[0.42325782658913...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[2.50065683339856...|[0.25006568333985...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[2.50065683339856...|[0.25006568333985...|       1.0|
|(24,[0,1,2,3,4,

In [142]:
predito_rfc_teste = modelo_rfc.transform(teste)

#Métricas

In [98]:
def calcula_matriz_confusao(df_transform_modelo, normalize = False, percentage = True):
  tp = df_transform_modelo.select('label','prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
  tn = df_transform_modelo.select('label','prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
  fp = df_transform_modelo.select('label','prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
  fn = df_transform_modelo.select('label','prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count()

  valorP = 1
  valorN = 1

  if normalize:
    valorP = tp + fn
    valorN = fp + tn

  if percentage and normalize:
    valorP = valorP / 100
    valorN = valorN / 100

  print(' '*20, 'Previsto')
  print(' '*4,'-'*3)
  print(' '*4,'|',' '*9, 'Churn', ' '*5 ,'Não-Churn')
  print(' '*4,'|','Churn', ' '*6, int(tp/valorP), ' '*7, int(fn/valorP))
  print('Real','|')
  print(' '*4,'|', 'Não-Churn', ' '*2, int(fp/valorN), ' '*7, int(tn/valorN))

In [112]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator()

In [113]:
def evaluator_ml(predito_ml):
  print("Acurácia: %f" % evaluator.evaluate(predito_ml, {evaluator.metricName: "accuracy"}))
  print("Precisão: %f" % evaluator.evaluate(predito_ml, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
  print("Recall: %f" % evaluator.evaluate(predito_ml, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
  print("F1: %f" % evaluator.evaluate(predito_ml, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

In [144]:
def print_modelo_metrica(modelo,predito_ml, predito_ml_teste):
  print('\n              ',modelo,' \n')
  print(50*('-'))
  print('\n Dados de Treino \n')
  print(50*('-'))
  print('  Métricas')
  print(50*('-'),'\n')
  evaluator_ml(predito_ml)
  print('\n')
  print(50*('-'))
  print('Matriz de Confusão')
  print(50*('-'))
  calcula_matriz_confusao(predito_ml, normalize=False)
  print('\n',50*('-'))
  print('\n Dados de Teste \n')
  print(50*('-'))
  print('  Métricas')
  print(50*('-'),'\n')
  evaluator_ml(predito_ml_teste)
  print('\n')
  print(50*('-'))
  print('Matriz de Confusão')
  print(50*('-'))
  calcula_matriz_confusao(predito_ml_teste, normalize=False)

In [145]:
print_modelo_metrica('REGRESSÃO LOGÍSTICA',predito_lr,predito_lr_teste)


               REGRESSÃO LOGÍSTICA  

--------------------------------------------------

 Dados de Treino 

--------------------------------------------------
  Métricas
-------------------------------------------------- 

Acurácia: 0.792086
Precisão: 0.774160
Recall: 0.828078
F1: 0.800212


--------------------------------------------------
Matriz de Confusão
--------------------------------------------------
                     Previsto
     --------------------------------
     |           Churn       Não-Churn
     | Churn        3020         627
Real |
     | Não-Churn    881         2725

 --------------------------------------------------

 Dados de Teste 

--------------------------------------------------
  Métricas
-------------------------------------------------- 

Acurácia: 0.771244
Precisão: 0.751073
Recall: 0.802227
F1: 0.775807


--------------------------------------------------
Matriz de Confusão
--------------------------------------------------
                  

In [146]:
print_modelo_metrica('ARVORE DE DECISÃO',predito_dtc,predito_dtc_teste)



               ARVORE DE DECISÃO  

--------------------------------------------------

 Dados de Treino 

--------------------------------------------------
  Métricas
-------------------------------------------------- 

Acurácia: 0.786020
Precisão: 0.773856
Recall: 0.811626
F1: 0.792291


--------------------------------------------------
Matriz de Confusão
--------------------------------------------------
                     Previsto
     --------------------------------
     |           Churn       Não-Churn
     | Churn        2960         687
Real |
     | Não-Churn    865         2741

 --------------------------------------------------

 Dados de Teste 

--------------------------------------------------
  Métricas
-------------------------------------------------- 

Acurácia: 0.774152
Precisão: 0.755556
Recall: 0.801572
F1: 0.777884


--------------------------------------------------
Matriz de Confusão
--------------------------------------------------
                    

In [147]:
print_modelo_metrica('RANDOM FOREST CLASSIFIER',predito_rfc,predito_rfc_teste)


               RANDOM FOREST CLASSIFIER  

--------------------------------------------------

 Dados de Treino 

--------------------------------------------------
  Métricas
-------------------------------------------------- 

Acurácia: 0.799118
Precisão: 0.774161
Recall: 0.847820
F1: 0.809318


--------------------------------------------------
Matriz de Confusão
--------------------------------------------------
                     Previsto
     --------------------------------
     |           Churn       Não-Churn
     | Churn        3092         555
Real |
     | Não-Churn    902         2704

 --------------------------------------------------

 Dados de Teste 

--------------------------------------------------
  Métricas
-------------------------------------------------- 

Acurácia: 0.783522
Precisão: 0.757977
Recall: 0.824492
F1: 0.789837


--------------------------------------------------
Matriz de Confusão
--------------------------------------------------
             

#Cross Validation

##Regressão Logística

In [163]:
grid = paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 0.5]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

evaluator = MulticlassClassificationEvaluator()

lr_cv = CrossValidator(
    estimator=lr,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed = seed
)


modelo_lr_cv = lr_cv.fit(treino)

In [164]:
previsoes_lr_cv_teste = modelo_lr_cv.transform(teste)

In [165]:
print('  Métricas')
print(50*('-'),'\n')
evaluator_ml(previsoes_lr_cv_teste)
print('\n')
print(50*('-'))
print('Matriz de Confusão')
print(50*('-'))
calcula_matriz_confusao(previsoes_lr_cv_teste, normalize=False)

  Métricas
-------------------------------------------------- 

Acurácia: 0.770921
Precisão: 0.747279
Recall: 0.809430
F1: 0.777114


--------------------------------------------------
Matriz de Confusão
--------------------------------------------------
                     Previsto
     --------------------------------
     |           Churn       Não-Churn
     | Churn        1236         291
Real |
     | Não-Churn    418         1150


##Arvore de Decisão

In [149]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

grid = ParamGridBuilder() \
    .addGrid(dtc.maxDepth, [2,5, 10]) \
    .addGrid(dtc.maxBins, [10, 32, 45]) \
    .build()

evaluator = MulticlassClassificationEvaluator()


In [151]:
dtc_cv = CrossValidator(
    estimator=dtc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed = seed
)

In [152]:
modelo_dtc_cv = dtc_cv.fit(treino)

In [153]:
previsoes_dtc_cv_teste = modelo_dtc_cv.transform(teste)

In [154]:
print('  Métricas')
print(50*('-'),'\n')
evaluator_ml(previsoes_dtc_cv_teste)
print('\n')
print(50*('-'))
print('Matriz de Confusão')
print(50*('-'))
calcula_matriz_confusao(previsoes_dtc_cv_teste, normalize=False)

  Métricas
-------------------------------------------------- 

Acurácia: 0.794507
Precisão: 0.759767
Recall: 0.853307
F1: 0.803825


--------------------------------------------------
Matriz de Confusão
--------------------------------------------------
                     Previsto
     --------------------------------
     |           Churn       Não-Churn
     | Churn        1303         224
Real |
     | Não-Churn    412         1156


## Random Forest

In [156]:
grid = ParamGridBuilder()\
        .addGrid(rfc.maxDepth, [2, 5, 10])\
        .addGrid(rfc.maxBins, [10, 32, 45])\
        .addGrid(rfc.numTrees, [10, 20, 50])\
        .build()

In [157]:
evaluator = MulticlassClassificationEvaluator()

In [159]:
rfc_cv = CrossValidator(
    estimator=rfc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=seed
)

In [160]:
modelo_rfc_cv = rfc_cv.fit(treino)

In [161]:
previsoes_rfc_cv_teste = modelo_rfc_cv.transform(teste)

In [162]:
print('  Métricas')
print(50*('-'),'\n')
evaluator_ml(previsoes_rfc_cv_teste)
print('\n')
print(50*('-'))
print('Matriz de Confusão')
print(50*('-'))
calcula_matriz_confusao(previsoes_rfc_cv_teste, normalize=False)

  Métricas
-------------------------------------------------- 

Acurácia: 0.805170
Precisão: 0.782051
Recall: 0.838900
F1: 0.809479


--------------------------------------------------
Matriz de Confusão
--------------------------------------------------
                     Previsto
     --------------------------------
     |           Churn       Não-Churn
     | Churn        1281         246
Real |
     | Não-Churn    357         1211


****
****
# <font color=red size=8>Conclusão: Melhor Modelo <font>

Dada a meta da empresa de manter uma base de clientes estável e evitar o cancelamento, os falsos negativos tornam-se mais críticos. Isso ocorre quando o modelo não prevê o churn, mas o cliente acaba cancelando, resultando na perda da oportunidade de tomar medidas para reter o cliente. Nesse contexto específico, em que é fundamental evitar a perda de clientes, optamos por utilizar um modelo de árvore de decisão. A razão para essa escolha reside no foco prioritário no Recall, que se torna de suma importância. O Recall destaca-se, neste momento, como uma métrica crucial, superando a precisão. Embora a Precisão tenha sido melhor na Random Forest, a decisão foi direcionada pelo valor crítico atribuído ao Recall, alinhando-se com nossa estratégia de retenção de clientes.

In [166]:
melhor_modelo_dtc_cv = modelo_rfc_cv.bestModel

print(melhor_modelo_dtc_cv.getMaxDepth())
print(melhor_modelo_dtc_cv.getMaxBins())


10
45


In [175]:
dtc_tunning = DecisionTreeClassifier(maxDepth=10, maxBins=45, seed=seed)
modelo_dtc_tunning = dtc_tunning.fit(dadosF)

###Novo Cliente

In [168]:
novo_cliente = [{
    'Mais65anos': 0,
    'MesesDeContrato': 1,
    'MesesCobrados': 45.30540797610398,
    'Conjuge': 0,
    'Dependentes': 0,
    'TelefoneFixo': 0,
    'MaisDeUmaLinhaTelefonica': 0,
    'SegurancaOnline': 0,
    'BackupOnline': 0,
    'SeguroDispositivo': 0,
    'SuporteTecnico': 0,
    'TVaCabo': 1,
    'StreamingFilmes': 1,
    'ContaCorreio': 1,
    'Internet_DSL': 1,
    'Internet_FibraOptica': 0,
    'Internet_Nao': 0,
    'TipoContrato_Mensalmente': 1,
    'TipoContrato_UmAno': 0,
    'TipoContrato_DoisAnos': 0,
    'MetodoPagamento_DebitoEmConta': 0,
    'MetodoPagamento_CartaoCredito': 0,
    'MetodoPagamento_BoletoEletronico': 1,
    'MetodoPagamento_Boleto': 0
}]

In [169]:
novo_cliente = spark.createDataFrame(novo_cliente)
novo_cliente.show()

+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+-----------------+---------------+----------------------+--------------------------------+-----------------------------+-----------------------------+---------------+-----------------+---------------+--------------+-------+------------+---------------------+------------------------+------------------+
|BackupOnline|Conjuge|ContaCorreio|Dependentes|Internet_DSL|Internet_FibraOptica|Internet_Nao|Mais65anos|MaisDeUmaLinhaTelefonica|    MesesCobrados|MesesDeContrato|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|SegurancaOnline|SeguroDispositivo|StreamingFilmes|SuporteTecnico|TVaCabo|TelefoneFixo|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|
+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+----

In [173]:
assembler = VectorAssembler(inputCols = X, outputCol = 'features')
novo_cliente_F = assembler.transform(novo_cliente).select('features')
novo_cliente_F.show(truncate=False)

+----------------------------------------------------------------------------+
|features                                                                    |
+----------------------------------------------------------------------------+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])|
+----------------------------------------------------------------------------+



In [176]:
modelo_dtc_tunning.transform(novo_cliente_F).show()

+--------------------+--------------+--------------------+----------+
|            features| rawPrediction|         probability|prediction|
+--------------------+--------------+--------------------+----------+
|(24,[1,2,11,12,13...|[138.0,1412.0]|[0.08903225806451...|       1.0|
+--------------------+--------------+--------------------+----------+



A coluna 'prediction' contém a informação crucial que estávamos procurando. Um valor de 1.0 indica que nosso modelo prevê que esse cliente está propenso a cancelar o serviço. Em outras palavras, esse é um cliente ao qual priorizaríamos, sugerindo ao time de marketing que ofereça promoções especiais para tentar retê-lo